In [1]:
import pandas as pd

df = pd.read_csv("pl-tables-1993-2024.csv")
df['season_start_year'] = df['season_end_year'] - 1

q1 = df[df['position'] == 1]['team'].value_counts().head(1)
print(f"Most 1st place finishes: {q1.index[0]} ({q1.values[0]} times)\n")

df_sorted = df.sort_values(['team', 'season_end_year'])
df_sorted['season_diff'] = df_sorted.groupby('team')['season_end_year'].diff()
df_sorted['consecutive_block'] = (df_sorted['season_diff'] != 1).cumsum()

df_sorted['gf_rolling_3yr'] = (df_sorted.groupby(['team', 'consecutive_block'], group_keys=False)
                             ['gf'].rolling(window=3, min_periods=3).sum()
                             .reset_index(level=[0,1], drop=True))
df_sorted['ga_rolling_3yr'] = (df_sorted.groupby(['team', 'consecutive_block'], group_keys=False)
                             ['ga'].rolling(window=3, min_periods=3).sum()
                             .reset_index(level=[0,1], drop=True))

max_gf = df_sorted.loc[df_sorted['gf_rolling_3yr'].idxmax()]
print(f"Best 3-year attack: {max_gf['team']}")
print(f"Seasons: {max_gf['season_start_year']}-{max_gf['season_end_year']}")
print(f"Total goals: {max_gf['gf_rolling_3yr']:.0f}\n")

min_ga = df_sorted.loc[df_sorted['ga_rolling_3yr'].idxmin()]
print(f"Best 3-year defense: {min_ga['team']}")
print(f"Seasons: {min_ga['season_start_year']}-{min_ga['season_end_year']}")
print(f"Goals conceded: {min_ga['ga_rolling_3yr']:.0f}\n")

valid_teams = df_sorted.groupby(['team', 'consecutive_block']).size()
valid_teams = valid_teams[valid_teams >= 3].index
df_valid = df_sorted.set_index(['team', 'consecutive_block']).loc[valid_teams].reset_index()

min_gf = df_valid.loc[df_valid['gf_rolling_3yr'].idxmin()]
print(f"Worst 3-year attack: {min_gf['team']}")
print(f"Seasons: {min_gf['season_start_year']}-{min_gf['season_end_year']}")
print(f"Total goals: {min_gf['gf_rolling_3yr']:.0f}\n")

max_ga = df_valid.loc[df_valid['ga_rolling_3yr'].idxmax()]
print(f"Worst 3-year defense: {max_ga['team']}")
print(f"Seasons: {max_ga['season_start_year']}-{max_ga['season_end_year']}")
print(f"Goals conceded: {max_ga['ga_rolling_3yr']:.0f}\n")

df_recent = df[df['season_end_year'].between(2023, 2024)]
pos_2023 = df_recent[df_recent['season_end_year'] == 2023][['team', 'position']].set_index('team')
pos_2024 = df_recent[df_recent['season_end_year'] == 2024][['team', 'position']].set_index('team')
deterioration = pos_2023.join(pos_2024, lsuffix='_2023', rsuffix='_2024').dropna()
deterioration = deterioration[deterioration['position_2024'] <= 17]
deterioration['change'] = deterioration['position_2024'] - deterioration['position_2023']
worst = deterioration.sort_values('change', ascending=False).head(1)
print(f"Biggest decline (2023-2024): {worst.index[0]}")
print(f"2023 position: {worst['position_2023'].values[0]}")
print(f"2024 position: {worst['position_2024'].values[0]}")
print(f"Position change: +{worst['change'].values[0]:.0f}\n")

brentford = df[(df['season_end_year'] == 2024) & (df['team'] == 'Brentford')]
brentford_position = brentford['position'].values[0]


higher_teams = df[(df['season_end_year'] == 2024) & 
                (df['position'] < brentford_position) & 
                (df['position'] >= brentford_position - 5)]

print("Brentford 2024 Performance:")
print(f"Position: {brentford_position}")
print(f"Goals scored: {brentford['gf'].values[0]}")
print(f"Goals conceded: {brentford['ga'].values[0]}")
print(f"Goal difference: {brentford['gd'].values[0]}\n")

print("Comparison with 5 Teams Above Brentford:")
print(f"Average goals scored: {higher_teams['gf'].mean():.1f} (Brentford: {brentford['gf'].values[0]})")
print(f"Average goals conceded: {higher_teams['ga'].mean():.1f} (Brentford: {brentford['ga'].values[0]})")
print(f"Average goal difference: {higher_teams['gd'].mean():.1f} (Brentford: {brentford['gd'].values[0]})\n")

print("Key Insights:")
print(f"1. Brentford scored {brentford['gf'].values[0] - higher_teams['gf'].mean():+.1f} goals more than these mid-table rivals")
print(f"2. But conceded {brentford['ga'].values[0] - higher_teams['ga'].mean():+.1f} more goals")

Most 1st place finishes: Manchester Utd (13 times)

Best 3-year attack: Manchester City
Seasons: 2019-2020
Total goals: 303

Best 3-year defense: Chelsea
Seasons: 2006-2007
Goals conceded: 61

Worst 3-year attack: Sunderland
Seasons: 2002-2003
Total goals: 96

Worst 3-year defense: Leeds United
Seasons: 2022-2023
Goals conceded: 211

Biggest decline (2023-2024): Brentford
2023 position: 9
2024 position: 16.0
Position change: +7

Brentford 2024 Performance:
Position: 16
Goals scored: 56
Goals conceded: 65
Goal difference: -9

Comparison with 5 Teams Above Brentford:
Average goals scored: 50.8 (Brentford: 56)
Average goals conceded: 61.2 (Brentford: 65)
Average goal difference: -10.4 (Brentford: -9)

Key Insights:
1. Brentford scored +5.2 goals more than these mid-table rivals
2. But conceded +3.8 more goals
